# 4 Pre-Processing and Training Data<a id='4_Pre-Processing_and_Training_Data'></a>

## 4.1 Contents

## 4.2 Introduction

## 4.3 Imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
# import seaborn as sns
# from sklearn import __version__ as sklearn_version
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingRegressor
from sklearn.metrics import r2_score, mean_absolute_error

from pycaret.regression import *
import os, glob
from statsmodels.tsa.stattools import acf

import datetime
%matplotlib inline  

## 4.4 Load Data

In [ ]:
train_df212 = pd.read_pickle('../data/train_df212.pkl')
# delete some features with small number of unique values
train_df212.drop(columns = ['median'], inplace = True)
train_df212.drop(columns = [col for col in train_df212.columns if '_median_smedian' in col], inplace = True)
train_df212

In [ ]:
train_df243 = pd.read_pickle('../data/train_df243.pkl')
train_df243.head()

In [ ]:
cols_to_use = train_df243.columns.difference(train_df212.columns)
train_df = pd.merge(train_df212, train_df243[cols_to_use], left_index=True, right_index=True, how='outer')
train_df.head()

In [ ]:
# print(train_df212.shape)
# print(train_df243.shape)
# newtrain_df = pd.concat([train_df212, train_df243.drop(columns = 'ttf')], axis = 1)
# train_df = newtrain_df.T.drop_duplicates().T
train_df.to_pickle('../data/train_df388.pkl')

In [4]:
train_df = pd.read_pickle('../data/train_df388.pkl')
train_df.shape
# columns = train_df.columns
# newtrain_df[columns]

(8355, 389)

In [5]:
col_todrop = list(train_df.T[train_df.T.duplicated()].index)
col_todrop

['rol1000_mean_iqr',
 'rol1000_mean_skew',
 'rol100_mean_skew',
 'rol10_mean_skew']

In [6]:
train_df.drop(columns = col_todrop, inplace = True)
train_df.shape

(8355, 385)

## 4.6 Auto ML using Pycaret

In [7]:
# init setup

# cv = LeaveOneGroupOut()
s = setup(train_df,
          target = 'ttf', 
          normalize = True,          
          session_id = 123,
          pca = True,
          pca_components = 0.95
         ) #event_df['event_id'] , fold_group100

,Description,Value
0,Session id,123
1,Target,ttf
2,Target type,Regression
3,Data shape,"(8355, 47)"
4,Train data shape,"(5848, 47)"
5,Test data shape,"(2507, 47)"
6,Numeric features,384
7,Preprocess,True
8,Imputation type,simple
9,Numeric imputation,mean


In [8]:
# model training and selection
best = compare_models(sort = 'MAE', exclude = ['par','lar','llar'])

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,2.0468,6.9487,2.6341,0.4792,0.5098,7.2512,6.7850
rf,Random Forest Regressor,2.0589,7.0239,2.6486,0.4732,0.5118,7.4425,13.7510
lightgbm,Light Gradient Boosting Machine,2.0658,7.0936,2.6623,0.4677,0.5060,7.3628,0.8580
gbr,Gradient Boosting Regressor,2.0807,7.0971,2.6627,0.4676,0.5121,7.4061,5.9770
knn,K Neighbors Regressor,2.0810,7.4534,2.7289,0.4402,0.5134,6.5265,0.3330
huber,Huber Regressor,2.1137,7.5149,2.7404,0.4356,0.5217,7.4851,0.3080
br,Bayesian Ridge,2.1356,7.4077,2.7207,0.4439,0.5283,7.5262,0.2130
lr,Linear Regression,2.1368,7.4117,2.7214,0.4436,0.5287,7.5211,1.7980
ridge,Ridge Regression,2.1368,7.4117,2.7214,0.4436,0.5287,7.5211,0.1800
omp,Orthogonal Matching Pursuit,2.1919,7.8858,2.8066,0.4085,0.5421,7.5921,0.2100


In [ ]:
# analyze best model
# fast result: Hyperparameters, Residuals, Prediction Error, Cooks Distance, Feature Importance, Interactive Residuals
# evaluate_model(best)

## 4.7 Create models using pycaret

### 4.7.1 Light Gradient Boosting Machine

In [9]:
lightgbm = create_model('lightgbm')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2.0049,6.9847,2.6429,0.4759,0.4681,3.6839
1,1.9848,6.4362,2.5370,0.4514,0.5068,3.2543
2,2.0794,7.1011,2.6648,0.5185,0.4896,7.3250
3,2.0183,6.6201,2.5729,0.4444,0.4758,6.3830
4,2.0217,6.5825,2.5656,0.4516,0.4754,3.8046
5,2.0784,7.3863,2.7178,0.4656,0.5216,3.1349
6,2.0888,7.2578,2.6940,0.4715,0.5281,3.6181
7,2.1141,7.5358,2.7451,0.4580,0.5271,6.1416
8,2.1337,7.5090,2.7402,0.4822,0.5302,2.6265


### 4.7.2 Random Forest

In [10]:
rf = create_model('rf')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2.0031,6.7537,2.5988,0.4933,0.4719,3.6188
1,1.9309,6.2685,2.5037,0.4657,0.5061,3.5052
2,2.0620,6.8887,2.6246,0.5329,0.4925,7.2827
3,1.9895,6.4351,2.5367,0.4599,0.4763,6.8373
4,2.0254,6.5301,2.5554,0.4560,0.4852,3.8398
5,2.0810,7.3447,2.7101,0.4687,0.5282,3.4494
6,2.0831,7.1821,2.6799,0.4770,0.5333,3.5645
7,2.1143,7.5045,2.7394,0.4603,0.5345,6.2702
8,2.1450,7.5385,2.7456,0.4802,0.5396,2.8001


### 4.7.3 Extra Trees Regressor

In [11]:
et = create_model('et')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1.9642,6.5235,2.5541,0.5105,0.4693,3.5912
1,1.9020,6.0668,2.4631,0.4829,0.4973,3.4478
2,2.0888,7.0132,2.6482,0.5245,0.5004,7.5812
3,1.9882,6.4119,2.5322,0.4619,0.4756,6.7728
4,1.9936,6.4081,2.5314,0.4661,0.4799,3.8300
5,2.0790,7.3313,2.7076,0.4696,0.5286,3.3481
6,2.0834,7.1727,2.6782,0.4777,0.5324,3.5771
7,2.1089,7.4800,2.7350,0.4620,0.5349,6.2872
8,2.1186,7.3721,2.7152,0.4917,0.5307,2.7320


### 4.7.4 Gradient Boosting Regressor

In [12]:
gbr = create_model('gbr')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2.0023,6.8068,2.6090,0.4893,0.4735,3.5199
1,2.0042,6.4991,2.5493,0.4460,0.5165,3.5466
2,2.1084,7.0831,2.6614,0.5197,0.4922,7.6741
3,2.0049,6.4947,2.5485,0.4549,0.4803,6.6396
4,2.0305,6.5685,2.5629,0.4528,0.4882,3.9832
5,2.0993,7.4868,2.7362,0.4584,0.5278,3.3891
6,2.1280,7.2734,2.6969,0.4704,0.5350,3.5973
7,2.1317,7.5002,2.7387,0.4606,0.5310,6.1700
8,2.1319,7.5202,2.7423,0.4815,0.5313,2.7919


## 4.8 Tune a Model

### 4.8.1 Light Gradient Boosting Machine

In [13]:
tuned_lightgbm = tune_model(lightgbm, optimize = 'MAE')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1.9890,6.7585,2.5997,0.4929,0.4684,3.5722
1,1.9878,6.4158,2.5329,0.4531,0.5099,3.4760
2,2.0915,7.0538,2.6559,0.5217,0.4941,7.4058
3,2.0187,6.5962,2.5683,0.4464,0.4772,6.3609
4,1.9983,6.4035,2.5305,0.4665,0.4810,3.9499
5,2.0942,7.4807,2.7351,0.4588,0.5275,3.3142
6,2.0968,7.1655,2.6768,0.4782,0.5312,3.5671
7,2.0920,7.3152,2.7047,0.4739,0.5218,6.4990
8,2.1072,7.4046,2.7211,0.4894,0.5301,2.6933


Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [ ]:
#tuned model object is stored in the variable 'tuned_lightgbm'. 
print(tuned_lightgbm)

In [ ]:
plot_model(tuned_lightgbm, plot='feature')

Q: Why the feature importance is so high in lightgbm model? What does it indicate?

In [ ]:
plot_model(tuned_lightgbm)

In [ ]:
plot_model(tuned_lightgbm, plot = 'error')

In [ ]:
pred_holdout = predict_model(tuned_lightgbm)

### 4.8.2 Random Forest

In [14]:
tuned_rf = tune_model(rf, optimize = 'MAE')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2.0331,6.8644,2.6200,0.4850,0.4824,3.4740
1,1.9683,6.2807,2.5061,0.4647,0.5160,3.5298
2,2.1214,7.1933,2.6820,0.5123,0.5102,7.4052
3,2.0269,6.5278,2.5550,0.4521,0.4849,6.6958
4,2.0416,6.5478,2.5589,0.4545,0.4947,3.7888
5,2.1440,7.6061,2.7579,0.4497,0.5426,3.3392
6,2.1355,7.2655,2.6955,0.4709,0.5463,3.5712
7,2.1497,7.6134,2.7592,0.4524,0.5453,6.0892
8,2.1988,7.8201,2.7964,0.4608,0.5506,2.7930


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [ ]:
plot_model(tuned_rf, plot='feature')

In [ ]:
plot_model(tuned_rf)

In [ ]:
plot_model(tuned_rf, plot = 'error')

In [ ]:
pred_holdout = predict_model(tuned_rf)

### 4.8.3 Extra Trees Regressor

In [ ]:
tuned_et = tune_model(et, optimize = 'MAE')

In [ ]:
plot_model(tuned_et, plot='feature')

In [ ]:
plot_model(tuned_et)

Q: Is it a good idea to dismiss Extra Tree Regressor? It is heavily over fitted, even though it has top 3 score.

In [ ]:
plot_model(tuned_et, plot = 'error')

In [ ]:
pred_holdout = predict_model(tuned_et)

### 4.8.4 Gradient Boosting Regressor

In [15]:
tuned_gbr = tune_model(gbr, optimize = 'MAE')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2.0983,7.4263,2.7251,0.4428,0.4857,3.4003
1,2.0214,6.7110,2.5906,0.4280,0.5139,3.3166
2,2.1580,7.7038,2.7756,0.4777,0.5028,7.0341
3,2.0224,6.5995,2.5689,0.4461,0.4811,6.3333
4,2.0867,7.1957,2.6825,0.4005,0.4923,4.6769
5,2.1645,7.8060,2.7939,0.4353,0.5299,3.0853
6,2.1100,7.4471,2.7289,0.4577,0.5357,3.8993
7,2.1400,7.6728,2.7700,0.4482,0.5420,6.2895
8,2.2123,7.9033,2.8113,0.4550,0.5406,2.6326


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [ ]:
plot_model(tuned_gbr, plot='feature')

In [ ]:
plot_model(tuned_gbr)

In [ ]:
plot_model(tuned_gbr, plot = 'error')

In [ ]:
pred_holdout = predict_model(tuned_gbr)

## 4.9 Voting Regressor

### 4.9.1 no weights

In [16]:
# exp_name = setup(data = boston,  target = 'medv')
top3 = compare_models(include = [tuned_lightgbm, tuned_rf,  tuned_gbr],n_select = 3,sort = 'MAE')
# top4 = compare_models(include = ['lightgbm', 'rf', 'et', 'gbr'],n_select = 4)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
1,Random Forest Regressor,2.0589,7.0239,2.6486,0.4732,0.5118,7.4425,16.3160
0,Light Gradient Boosting Machine,2.0639,7.0315,2.6504,0.4725,0.5085,7.3306,1.3820
2,Gradient Boosting Regressor,2.0807,7.0971,2.6627,0.4676,0.5121,7.4061,5.5450


In [17]:
blender = blend_models(top3, optimize = 'MAE')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1.9848,6.6751,2.5836,0.4992,0.4685,3.5674
1,1.9531,6.2712,2.5042,0.4655,0.5073,3.5048
2,2.0774,6.9021,2.6272,0.5320,0.4897,7.4512
3,1.9863,6.4003,2.5299,0.4628,0.4746,6.6086
4,2.0048,6.4009,2.5300,0.4667,0.4814,3.9209
5,2.0784,7.3258,2.7066,0.4700,0.5246,3.3811
6,2.0789,7.0810,2.6610,0.4844,0.5281,3.5538
7,2.0977,7.3299,2.7074,0.4728,0.5266,6.3096
8,2.1094,7.3645,2.7138,0.4922,0.5308,2.7572


In [ ]:
pred_holdout = predict_model(blender)

# print(mean_absolute_error(y_test,pred_test['prediction_label']))
# print(r2_score(y_test,pred_test['prediction_label']))

In [ ]:
print(blender)

In [ ]:
plot_model(blender, plot='pipeline')

### 4.9.2 Find best weights using VotingRegressor()

In [ ]:
y = train_df["ttf"]
X = train_df.drop(columns = ["ttf" ])
feature_names=X.columns

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=47)


In [ ]:
top03_copy = top3

In [ ]:
# use sklearn vote weights
# Create empty lists that will storage the different weights

weights1 = []
weights2 = []
weights3 = []
scores = []

# Create a for loop to evaluate different combinations of weights

for i in np.arange(0.1,1, 0.1):
    now = datetime.datetime.now()
    print(i, now)
    for j in np.arange(0.1,1, 0.1):
        
#         for k in np.arange(0.1,1, 0.1):
        k = 1 - i - j
        if k > 0:            
            rg_voting3 = VotingRegressor(estimators = [('est1', top03_copy[0]), ('est2', top03_copy[1]), 
                                           ('est3', top03_copy[2])], n_jobs = -1,weights = [i, j, k])
            rg_votine3.fit(X_train, y_train)
            pred = rg_voting3.predict(X_test)
            score = mean_absolute_error(y_test, pred)
            scores.append(score)
            weights1.append(i)
            weights2.append(j)
            weights3.append(k)

In [ ]:
# Save the results in a data frame

test_scores03 = pd.DataFrame()
test_scores03['Weight1'] = weights1
test_scores03['Weight2'] = weights2
test_scores03['Weight3'] = weights3
test_scores03['Test MAE'] = scores

condition = (test_scores03['Weight3'] >=0.01)

test_scores03 = test_scores03.loc[condition]
# Sort the values to see the different test scores depending on the weights
test_scores03.sort_values(by = 'Test MAE', ascending = True).head(15)

In [ ]:
rg_voting03 = VotingRegressor(estimators = [('est1', top03_copy[0]), ('est2', top03_copy[1]), 
                                           ('est3', top03_copy[2])], n_jobs = -1,weights = test_scores03.sort_values(by = 'Test MAE', ascending = True).iloc[0,:3])
#Fit and predict

rg_voting03.fit(X_train, y_train)
pred_voting03 = rg_voting.predict(X_test)

In [ ]:
rg_voting03

In [ ]:
mean_absolute_error(pred_voting03,y_test)

In [ ]:
top3_copy = top3

In [ ]:
# use sklearn vote weights
# Create empty lists that will storage the different weights

weights1 = []
weights2 = []
weights3 = []
scores = []

# Create a for loop to evaluate different combinations of weights

for i in np.arange(0.1,1, 0.1):
    now = datetime.datetime.now()
    print(i, now)
    for j in np.arange(0.1,1, 0.1):
        
#         for k in np.arange(0.1,1, 0.1):
        k = 1 - i - j
        if k > 0:            
            rg_voting = VotingRegressor(estimators = [('est1', top3_copy[0]), ('est2', top3_copy[1]), 
                                           ('est3', top3_copy[2])], n_jobs = -1,weights = [i, j, k])
            rg_voting.fit(X_train, y_train)
            pred = rg_voting.predict(X_test)
            score = mean_absolute_error(y_test, pred)
            scores.append(score)
            weights1.append(i)
            weights2.append(j)
            weights3.append(k)

In [ ]:
# Save the results in a data frame

test_scores = pd.DataFrame()
test_scores['Weight1'] = weights1
test_scores['Weight2'] = weights2
test_scores['Weight3'] = weights3
test_scores['Test MAE'] = scores

condition = (test_scores['Weight3'] >=0.01)

test_scores = test_scores.loc[condition]
# Sort the values to see the different test scores depending on the weights
test_scores.sort_values(by = 'Test MAE', ascending = True).head(15)

In [ ]:
rg_voting = VotingRegressor(estimators = [('est1', top3_copy[0]), ('est2', top3_copy[1]), 
                                           ('est3', top3_copy[2])], n_jobs = -1,weights = test_scores.sort_values(by = 'Test MAE', ascending = True).iloc[0,:3])
#Fit and predict

rg_voting.fit(X_train, y_train)
pred_voting = rg_voting.predict(X_test)

In [ ]:
mean_absolute_error(pred_voting,y_test)

In [ ]:
rg_voting

## 4.10 Plot

### 4.10.1 Feature Importance Plot

In [ ]:
# plot_model(best_mae_model)

In [ ]:
# plot_model(best_mae_model, plot = 'error')

* When label is high, underestimated
* When label is low, overestimated
* Some labels with values close to zero are significantly overestimated

## 4.11 Predict on test / hold-out Sample

In [ ]:
# predict_model(best_mae_model);

## 4.12 Submit to Kaggle

In [ ]:
testfilepath = "../raw_data/test/"
os.chdir(testfilepath)
testfiles = glob.glob("*.csv")
seg_len = 150_000

Generate 243 features

In [ ]:
# # time-domain features
# qt5 = lambda x: x.quantile(0.05)
# qt25 = lambda x: x.quantile(0.25)
# qt75 = lambda x: x.quantile(0.75)
# qt95 = lambda x: x.quantile(0.95)
# iqr = lambda x : qt75(x) - qt25(x)
# statistics_names = ["mean","median","std","kurt","skew","min","max","qt5","qt25","qt75","qt95","iqr"]
# statistics_funs = ["mean","median","std","kurt","skew","min","max",qt5,qt25,qt75,qt95,iqr]

# rol_funcs_names = ["mean","std","kurt","skew"]
# windows = [10, 100,1000]
# rol_feature_names = ["rol" + str(w) + "_" + func1 + "_"+ func2 \
#                      for w in windows\
#                      for func1 in rol_funcs_names \
#                      for func2 in statistics_names \
#                      ]

# # envelope_features
# envelope_names = ["rol" + str(w) + "envelope_" + fun \
#                   for w in windows\
#                   for fun in ["mean","std","max","kurt","skew"]]

# # lag_features
# lag_names = ["lag" + str(i) for i in range(1,12)]

# tfeature_names = statistics_names + rol_feature_names + envelope_names + lag_names

# def chunk_statistics(chunk):
#     temp_np = chunk['acoustic_data'].agg(statistics_funs)
#     return temp_np.tolist()

# def rolling_statistics(chunk, windows):
#     rtn_list = []

#     for w in windows:
#         temp_rolling = chunk['acoustic_data'].rolling(w, center = True)
#         temp_df = temp_rolling.agg(rol_funcs_names).agg(statistics_funs)
#         temp_list = temp_df.values.ravel().tolist()
#         rtn_list = temp_list + rtn_list
#     return rtn_list     

# def rectify_statistics(chunk, windows):
#     envelope_stats_list = []
#     signal_rectified = chunk['acoustic_data'].apply(np.abs)
#     for w in windows:
#         signal_envelope = signal_rectified.rolling(w, center = True).mean()
#         signal_envelope_list = signal_envelope.agg(["mean","std","max","kurt","skew"]).tolist()
#         envelope_stats_list = envelope_stats_list + signal_envelope_list
#     return envelope_stats_list

# def ac_features(chunk):    
#     temp = acf(chunk['acoustic_data'], nlags=11, fft = True)
#     return temp[1:].tolist()

# def generate_stats(chunk,windows):
#     """ statistic feature in time_domain"""
#     rol_stats_list = rolling_statistics(chunk, windows)
#     stats_list = chunk_statistics(chunk)
#     envelope_stats_list = rectify_statistics(chunk, windows)
#     ac_list = ac_features(chunk)
#     tfeature_list = stats_list +  rol_stats_list + envelope_stats_list + ac_list
#     return tfeature_list

In [ ]:
# # frequency features
# # rolling statistic feature in time_domain
# freq_stat_names = ["mean","median","std","kurt","skew","min","max","qt25","qt75","iqr"]
# freq_windows = [5000, 10000, 50000] #
# amp_spectrum_rol_feature_names = ["amp_sp_rol" + str(w) + "_"+ freq_stat_name \
#                                   for w in freq_windows \
#                                   for freq_stat_name in freq_stat_names] 
# amp_spectrum_rol_seg_feature_names = ["amp_sp_rol" + str(w) + "seg_" + str(i) for w in freq_windows for i in range(10)]

# # rolling functions for amplitude spectrum
# def roll_seg_median(spectrum_np, seg_len):
#     interval = int(seg_len/20)
#     roll_spectrum_seg_median = []
#     for i in range(10):
#         idx = list((np.arange(0,interval) + interval * i))
#         roll_spectrum_seg_median.append(np.median(spectrum_np[idx]))

#     return roll_spectrum_seg_median

# def rolling_amp_spectrum(chunk, windows, seg_len):
#     roll_amp_spectrum_statistics = []
#     roll_amp_spectrum_seg_median= []
#     sp = np.fft.fft(chunk['acoustic_data'])
#     amp_sp = pd.Series(np.abs(sp))[0:int(seg_len/2)]
#     for w in windows:
#         temp_rolling_median = amp_sp.rolling(w, center = True).agg(np.median)
#         roll_amp_spectrum_statistics.append(temp_rolling_median.
#                                             agg(["mean","median","std","kurt","skew","min","max",qt25,qt75,iqr])) #   
#         roll_amp_spectrum_seg_median.append(roll_seg_median(temp_rolling_median, seg_len)) 
#     roll_amp_spectrum_statistics = [j for sub in roll_amp_spectrum_statistics for j in sub]        
#     roll_amp_spectrum_seg_median = [j for sub in roll_amp_spectrum_seg_median for j in sub]

#     return roll_amp_spectrum_statistics + roll_amp_spectrum_seg_median

# # amplitude spectrum trend (ac)
# def ac_spectrum(chunk):
#     sp = np.fft.fft(chunk['acoustic_data'])
#     SP = np.abs(sp)
#     ac_spectrum_list = acf(SP, nlags=11, fft = True)[1:]
#     return ac_spectrum_list


In [ ]:
# # spectrogram features
# import librosa as lr
# def spectrogram_feature(chunk, HOP_LENGTH, SIZE_WINDOW):    
#     spec = lr.stft(chunk['acoustic_data'].values.astype(float), hop_length = HOP_LENGTH, n_fft = SIZE_WINDOW)
#     spec_rectified = np.abs(spec)
#     mean_bandwidth = np.mean(lr.feature.spectral_bandwidth(S = spec_rectified))
#     mean_centroids = np.mean(lr.feature.spectral_centroid(S = spec_rectified))
#     return mean_centroids, mean_bandwidth


Generate 243 features

In [ ]:
# j = -1

# seg_id =[]
# tfeatures = []

# roll_AP_statistics = []
# ac_spectrum_list = []

# HOP_LENGTH = 2**6
# SIZE_WINDOW = 2**7
# bandwidths = []
# centroids = []

# for file in testfiles: 
#     chunk = pd.read_csv(file)
#     j += 1
#     if j % 500 == 0:
#         print(j)
#         now = datetime.datetime.now()
#         print(now)
    
#     tfeature_list = generate_stats(chunk, windows)
#     tfeatures.append( tfeature_list)

#     seg_id.append(file[:-4])

#     roll_amp_spectrum_statistics = rolling_amp_spectrum(chunk, freq_windows, seg_len)
#     roll_AP_statistics.append(roll_amp_spectrum_statistics)
    
#     ac_spectrum_row = ac_spectrum(chunk)
#     ac_spectrum_list.append(ac_spectrum_row)
    
#     mean_centroids, mean_bandwidth = spectrogram_feature(chunk, HOP_LENGTH, SIZE_WINDOW)
#     centroids.append(mean_centroids)
#     bandwidths.append(mean_bandwidth)
# #     if j > 2:
# #         break

# tfeature_df = pd.DataFrame(tfeatures, columns = tfeature_names)
# roll_AP_statistics_df = pd.DataFrame(roll_AP_statistics, columns = amp_spectrum_rol_feature_names +
#                                     amp_spectrum_rol_seg_feature_names)
# ac_amp_spec_df = pd.DataFrame(ac_spectrum_list, columns = ['amp_spec_ac' + str(i) for i in range(1,12)])
# spectrogram_feature_df = pd.DataFrame({'mean_centroids':centroids, 'bandwidth':bandwidths})



In [ ]:
# roll_AP_statistics_df.dropna(axis = 1, inplace = True)
# test_df = pd.concat([tfeature_df, 
#                      roll_AP_statistics_df, 
#                      ac_amp_spec_df, 
#                      spectrogram_feature_df], axis = 1) 
# test_df.index = pd.Index(seg_id)
# test_df.index.name = 'seg_id'
# test_df.to_pickle('../../data/test_features243.pkl')
# test_df.head()

Generate 212 features

In [ ]:
qt25 = lambda x: x.quantile(0.25)
qt75 = lambda x: x.quantile(0.75)
iqr = lambda x : qt75(x) - qt25(x)
abs_sum = lambda x : sum(abs(x))
statistics_names = ["mean","median","std","kurt","skew","min","max","qt25","qt75","iqr","abs_sum"]

rol_funcs = ["mean","median","std","kurt","skew"]
rol_funcs_names = ["mean","median","std","kurt","skew"]

windows = [50, 100,500,1000, 5000]
rol_feature_names = ["rol" + str(w) + "_" + func1 + "_"+ func2 \
                     for w in windows\
                     for func1 in rol_funcs_names \
                     for func2 in ["smean","smedian","sstd","skurt","sskew"] \
                     ]

rol10_feature_names = ["rol10" + "_" + func1 + "_"+ func2 \
                     for func1 in rol_funcs_names \
                     for func2 in ["smean","smedian","sstd","skurt","sskew"] \
                     ]

lag_names = ["lag" + str(i) for i in range(1,11)]

tfeature_names = statistics_names + rol_feature_names + lag_names
def chunk_statistics(chunk):
    temp_df = chunk['acoustic_data'].agg(["mean","median","std","kurt","skew","min","max",qt25,qt75,iqr,abs_sum])
    return pd.DataFrame(temp_df.values.reshape(1,len(statistics_names) ), columns = statistics_names)

# rol_features
def rolling_statistics(chunk, windows):
    rtn_df = []

    for w in windows:
        temp_rolling = chunk['acoustic_data'].rolling(w, center = True)
        temp_df = temp_rolling.agg(rol_funcs).agg(["mean","median","std","kurt","skew"])
        window_df = pd.DataFrame(temp_df.values.reshape((1,len(temp_df.columns) * len(temp_df))), 
                     columns = ["rol" + str(w) + "_" + func1 + "_"+ func2 \
                                for func2 in ["smean","smedian","sstd","skurt","sskew"] for func1 in rol_funcs_names ])
        if len(rtn_df) > 0:
            rtn_df = pd.concat([rtn_df, window_df], axis=1)
        else:
            rtn_df = window_df
    return rtn_df      
    

def ac_features(chunk):
    
    temp = acf(chunk['acoustic_data'], nlags=10, fft = True)
    return pd.DataFrame(temp[1:].reshape(1,10), columns = lag_names)

def generate_stats(chunk, windows):
    """ statistic feature in time_domain"""
    rol_stats_df = rolling_statistics(chunk, windows)
    stats_df = chunk_statistics(chunk)
    ac_df = ac_features(chunk)
    tfeature_list = pd.concat([stats_df, rol_stats_df,ac_df], axis = 1).values.tolist()[0] # rol_stats_df
    return tfeature_list


In [ ]:
j = -1


seg_id =[]
rol10 = []
tfeatures = []
# roll_AP_statistics = []

for file in testfiles: 
    chunk = pd.read_csv(file)
    j += 1
    if j % 500 == 0:
        print(j)
        now = datetime.datetime.now()
        print(now)
    
    tfeature_list = generate_stats(chunk, windows)
    tfeatures.append( tfeature_list)
    rol10.append(rolling_statistics(chunk, [10]).values.tolist()[0]) 

    seg_id.append(file[:-4])

#     roll_amp_spectrum_statistics = rolling_amp_spectrum(chunk, freq_windows, seg_len)
#     roll_AP_statistics.append(roll_amp_spectrum_statistics)
    
    if j > 2:
        break

tfeature_df = pd.DataFrame(tfeatures, columns = tfeature_names, index = seg_id)
trol10_features_df = pd.DataFrame(rol10, columns = rol10_feature_names, index = seg_id)
tfeatures_df = pd.concat([tfeature_df,trol10_features_df], axis = 1)
tfeatures_df.to_pickle('../../data/test_features_timedomain212.pkl')
# roll_AP_statistics_df = pd.DataFrame(roll_AP_statistics, columns = amp_spectrum_rol_feature_names +
#                                     amp_spectrum_rol_seg_feature_names)



In [ ]:
test_df243 = pd.read_pickle('../../data/test_features243.pkl')
roll_AP_statistics_df = test_df243[[col for col in test_df243.columns if 'amp_sp_rol' in col]]


In [ ]:
test_df = pd.concat([tfeatures_df, roll_AP_statistics_df ], axis = 1) 
test_df.index = pd.Index(seg_id)
test_df.index.name = 'seg_id'
test_df.drop(columns = ['median'], inplace = True)
test_df.drop(columns = [col for col in test_df.columns if '_median_smedian' in col], inplace = True)

test_df.to_pickle('../../data/test_features212.pkl')
test_df.head()

In [ ]:
test_df212 = pd.read_pickle('../data/test_features212.pkl')
test_df212.shape

### single model prediction

In [18]:
test_df384 = pd.read_pickle('../data/test_df384.pkl')

In [19]:
best3_models = [tuned_lightgbm, tuned_rf,  tuned_gbr]
model_names = ['lightgbm', 'rf','gbr']
for idx, single_model in enumerate(best3_models):     
#     single_model.fit(X,y)
    pred_test = predict_model(single_model ,data = test_df384)
    pred_label = pd.DataFrame(pred_test['prediction_label'].values, index = test_df384.index)
    pred_label.columns = ['time_to_failure']
    pred_label.to_csv('../data/submission_384_pca95' + model_names[idx] + '.csv')

### blend model prediction

In [20]:
# blender.fit(X,y)
pred_test = predict_model(blender ,data = test_df384)
pred_label = pd.DataFrame(pred_test['prediction_label'].values, index = test_df384.index)
pred_label.columns = ['time_to_failure']
pred_label.to_csv('../data/submission384_pca95_noweights_voting.csv')

### weighted voting regressor prediction

In [ ]:
# rg_voting.fit(X, y)
pred_voting = rg_voting.predict(test_df)
pred_label = pd.DataFrame(pred_voting, index = test_df.index)
pred_label.columns = ['time_to_failure']
pred_label.to_csv('../../data/submission_weighted_voting.csv')

Private Score on kaggle and rank using 212 features:
* blender no weights: 2.471, 114, <top3%
* gbr: 2.481, 150 
* lightgbr: 2.487
* VotingRegressor with weithts： 2.493
* rf: 2.495



Private Score on kaggle and rank using 243 features on 9/28/2022 is poorer, even though the trainig performance is better:
* blender no weights: 2.505
* gbr: 2.5889 
* lightgbr: 2.5152
* VotingRegressor with weithts： 2.5196
* rf: 2.5115

Private Score on kaggle and rank using 384 features on 9/29/2022 intermediate:

* blender no weights: 2.4932
* gbr: 2.57878
* lightgbr: 2.5017
* rf: 2.4968

Private Score on kaggle and rank using 384 features with pca 0.8 on 9/29/2022:

* blender no weights: 2.508
* gbr: 2.52355
* lightgbr: 2.4885
* rf: 2.54866